In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50
from keras import losses
 
import random
from keras import metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2
import PIL
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
from tensorflow.keras import layers, models

# Residual block
# def residual_block(input_tensor, num_filters):
#     x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
#     x = layers.BatchNormalization()(x)
#     x = layers.ReLU()(x)

#     x = layers.Conv2D(num_filters, 3, padding="same")(x)
#     x = layers.BatchNormalization()(x)

#     # Add the input_tensor (residual connection) to the output of the convolution block
#     x = layers.add([x, input_tensor])
#     x = layers.ReLU()(x)

#     return x
def residual_block(input_tensor, num_filters):
    # Check if a projection shortcut is needed
    input_shape = input_tensor.shape[-1]
    if input_shape != num_filters:
        shortcut = layers.Conv2D(num_filters, (1, 1), strides=(1, 1), padding='same')(input_tensor)
    else:
        shortcut = input_tensor

    x = layers.Conv2D(num_filters, 3, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(num_filters, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Add the shortcut (which has been adjusted if necessary) to the output of the convolution block
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)

    return x
# Attention gate
def attention_gate(input_tensor, gate_tensor, num_filters):
    # Resize gate tensor to match the input_tensor shape using a 1x1 convolution
    gate_resized = layers.Conv2D(num_filters, 1, padding='same')(gate_tensor)
    gate_resized = layers.BatchNormalization()(gate_resized)
    gate_resized = layers.ReLU()(gate_resized)

    # Add the gate to the input_tensor
    x = layers.add([input_tensor, gate_resized])
    attention = layers.Conv2D(num_filters, 1, padding='same', activation='sigmoid')(x)
    x = layers.multiply([input_tensor, attention])

    return x

def conv_block(input_tensor, num_filters):
    x = residual_block(input_tensor, num_filters)
    return x

def encoder_block(input_tensor, num_filters):
    x = conv_block(input_tensor, num_filters)
    p = layers.MaxPooling2D((2, 2))(x)
    return x, p

# Modify the decoder_block to include an attention gate
def decoder_block(input_tensor, skip_features, num_filters):
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input_tensor)

    # Apply attention gate to the skip features before concatenating
    attention_skipped = attention_gate(skip_features, x, num_filters)

    # Concatenate upsampled input with the attention modified skip features
    x = layers.concatenate([x, attention_skipped])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = layers.Input(input_shape)

    # Encoder
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    # Bridge
    b1 = conv_block(p4, 1024)

    # Decoder
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    # Output
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = models.Model(inputs, outputs)
    return model

input_shape = (112, 112, 3)
model = build_unet(input_shape)
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=[metrics.Precision(),
                            metrics.Recall(),
                            metrics.FalsePositives(),
                            metrics.FalseNegatives(),
                            metrics.BinaryIoU()])

In [4]:

# Paths to your data.
images_path = './archive/crack_segmentation_dataset/train/images/'
masks_path = './archive/crack_segmentation_dataset/train/masks/'
img_size = (112, 112)

# Load and preprocess data
def load_data(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    files = []
    count = 0
    for filename in os.listdir(images_path):
        if count > 500:
            break
        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename
        # print(img_path)
        # Load and preprocess the image
        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # print(img.size())
    #   if not img or not mask:
    #         continue
        img, mask = img/255.0, mask/255.0
        # img = cv2.resize(img, img_size) / 255.0
        # # Load and preprocess the mask using OpenCV
        # mask = cv2.resize(mask, img_size) / 255.0
 
        for i in range(4):
            for j in range(4):
                img_part = img[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                mask_part = mask[i * 112: (i + 1) * 112,
                            j * 112: (j + 1) * 112]
                images.append(img_part)
                masks.append(mask_part)
        files.append(filename)
         
        count += 1
    print(files)
    return np.array(images), np.array(masks)


In [19]:
def load_test(images_path, masks_path, img_size):
    images = []  # List to store the images
    masks = []   # List to store the masks
    path = ''
    count = 0
    print()
    imgList = os.listdir(images_path)
    files = []
    for i in range(len(imgList) - 1, -1, -1):
        filename = imgList[i]
    # for filename in os.listdir(images_path):
        if count >= 100:
            break

        img_path = images_path + filename
        mask_path = masks_path + filename  # Assuming mask has same filename

        img = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img, mask = img/255.0, mask/255.0
        while 1:
            i, j = random.randint(0, 3), random.randint(0, 3)
            img_part = img[i * 112: (i + 1) * 112,
                        j * 112: (j + 1) * 112]
            mask_part = mask[i * 112: (i + 1) * 112,
                        j * 112: (j + 1) * 112]
            if np.sum(mask_part) > 0:
                break
        images.append(img_part)
        masks.append(mask_part)

        files.append(filename)
        count += 1
    print(files)
    return np.array(images), np.array(masks)

In [6]:
# Load your data
images, masks = load_data(images_path, masks_path, (112, 112))


['CFD_002.jpg', 'CFD_003.jpg', 'CFD_004.jpg', 'CFD_005.jpg', 'CFD_006.jpg', 'CFD_008.jpg', 'CFD_009.jpg', 'CFD_010.jpg', 'CFD_012.jpg', 'CFD_015.jpg', 'CFD_016.jpg', 'CFD_017.jpg', 'CFD_018.jpg', 'CFD_020.jpg', 'CFD_021.jpg', 'CFD_022.jpg', 'CFD_023.jpg', 'CFD_024.jpg', 'CFD_025.jpg', 'CFD_026.jpg', 'CFD_027.jpg', 'CFD_028.jpg', 'CFD_029.jpg', 'CFD_030.jpg', 'CFD_031.jpg', 'CFD_032.jpg', 'CFD_033.jpg', 'CFD_034.jpg', 'CFD_035.jpg', 'CFD_036.jpg', 'CFD_038.jpg', 'CFD_039.jpg', 'CFD_041.jpg', 'CFD_042.jpg', 'CFD_043.jpg', 'CFD_044.jpg', 'CFD_045.jpg', 'CFD_046.jpg', 'CFD_048.jpg', 'CFD_049.jpg', 'CFD_050.jpg', 'CFD_051.jpg', 'CFD_052.jpg', 'CFD_053.jpg', 'CFD_054.jpg', 'CFD_055.jpg', 'CFD_056.jpg', 'CFD_057.jpg', 'CFD_058.jpg', 'CFD_059.jpg', 'CFD_060.jpg', 'CFD_061.jpg', 'CFD_062.jpg', 'CFD_063.jpg', 'CFD_064.jpg', 'CFD_065.jpg', 'CFD_066.jpg', 'CFD_067.jpg', 'CFD_068.jpg', 'CFD_069.jpg', 'CFD_071.jpg', 'CFD_072.jpg', 'CFD_073.jpg', 'CFD_074.jpg', 'CFD_075.jpg', 'CFD_076.jpg', 'CFD_077.

In [7]:
images.shape

(8016, 112, 112, 3)

In [8]:
images1, masks1 = images[:2000], masks[:2000]

In [20]:
masks1 = np.expand_dims(masks1, axis=-1)
# Load your data
imgval, maskval = load_test(images_path, masks_path, (112, 112))

maskval = np.expand_dims(maskval, axis=-1)


['Volker_DSC01710_725_197_1392_1586.jpg', 'Volker_DSC01710_608_6_1524_1817.jpg', 'Volker_DSC01710_521_437_1123_892.jpg', 'Volker_DSC01710_449_161_1824_1502.jpg', 'Volker_DSC01710_37_518_1605_1304.jpg', 'Volker_DSC01710_340_159_2049_1635.jpg', 'Volker_DSC01710_15_28_1388_1132.jpg', 'Volker_DSC01710_112_246_1993_1548.jpg', 'Volker_DSC01710_1087_22_1502_1742.jpg', 'Volker_DSC01710_1025_76_1293_1327.jpg', 'Volker_DSC01710_0_0_2736_1824.jpg', 'Volker_DSC01709_599_1337_1214_1373.jpg', 'Volker_DSC01709_525_1036_1256_1448.jpg', 'Volker_DSC01709_43_382_1337_1425.jpg', 'Volker_DSC01709_1_428_1791_1806.jpg', 'Volker_DSC01709_191_287_1561_2056.jpg', 'Volker_DSC01709_176_298_1435_1635.jpg', 'Volker_DSC01709_14_103_1566_1955.jpg', 'Volker_DSC01709_109_399_1680_2130.jpg', 'Volker_DSC01709_0_0_1824_2736.jpg', 'Volker_DSC01708_820_247_911_1098.jpg', 'Volker_DSC01708_7_339_1805_2017.jpg', 'Volker_DSC01708_6_308_1788_2155.jpg', 'Volker_DSC01708_3_1033_1692_1349.jpg', 'Volker_DSC01708_26_342_1730_1845.jp

In [10]:
masks1.shape

(2000, 112, 112, 1)

In [11]:

X_train, y_train, X_val, y_val = images1, masks1,imgval, maskval 
# y_train.shape
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (2000, 112, 112, 3)
y_train shape: (2000, 112, 112, 1)
X_val shape: (100, 112, 112, 3)
y_val shape: (100, 112, 112, 1)


In [12]:
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()  # No augmentation for validation data

# Generate batches of augmented data from arrays (X_train and y_train)
train_generator = train_datagen.flow(X_train, y_train, batch_size=50, seed = 27)
val_generator = val_datagen.flow(X_val, y_val, batch_size=5, seed = 82)
 
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - (numerator + 1) / (denominator + 1)

model.compile(optimizer='adam',
                   loss=dice_loss, 
                   metrics=[metrics.Precision(),
                            metrics.Recall(),
                            metrics.FalsePositives(),
                            metrics.FalseNegatives(),
                            metrics.BinaryIoU()])
# Train the model using the fit_generator function
history = model.fit(
    train_generator, 
    steps_per_epoch= 21,  
    epochs=10, 
    validation_data=val_generator,
    validation_steps= 6 
)


Epoch 1/10


c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 176s 7s/step - binary_io_u_1: 0.4598 - false_negatives_1: 137251.2344 - false_positives_1: 740278.8125 - loss: 0.8132 - precision_1: 0.1173 - recall_1: 0.4372 - val_binary_io_u_1: 0.0263 - val_false_negatives_1: 300.0000 - val_false_positives_1: 348478.0000 - val_loss: 0.8996 - val_precision_1: 0.0602 - val_recall_1: 0.9867
Epoch 2/10
19/21 ━━━━━━━━━━━━━━━━━━━━ 13s 7s/step - binary_io_u_1: 0.5671 - false_negatives_1: 144522.8438 - false_positives_1: 150618.1562 - loss: 0.6662 - precision_1: 0.3111 - recall_1: 0.3304

c:\Users\link1\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


21/21 ━━━━━━━━━━━━━━━━━━━━ 127s 6s/step - binary_io_u_1: 0.5706 - false_negatives_1: 156849.4219 - false_positives_1: 157283.9531 - loss: 0.6545 - precision_1: 0.3210 - recall_1: 0.3336 - val_binary_io_u_1: 0.0139 - val_false_negatives_1: 3.0000 - val_false_positives_1: 357904.0000 - val_loss: 0.9203 - val_precision_1: 0.0487 - val_recall_1: 0.9998
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 155s 7s/step - binary_io_u_1: 0.6231 - false_negatives_1: 146824.5938 - false_positives_1: 90186.9062 - loss: 0.5423 - precision_1: 0.4975 - recall_1: 0.3797 - val_binary_io_u_1: 0.1532 - val_false_negatives_1: 9709.0000 - val_false_positives_1: 253987.0000 - val_loss: 0.8794 - val_precision_1: 0.0773 - val_recall_1: 0.6866
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 149s 7s/step - binary_io_u_1: 0.6290 - false_negatives_1: 154943.4219 - false_positives_1: 85514.1875 - loss: 0.5109 - precision_1: 0.5176 - recall_1: 0.3813 - val_binary_io_u_1: 0.1798 - val_false_negatives_1: 3033.0000 - val_false_positives_1: 7

In [13]:
print( len(val_generator) )

20


In [14]:

# Load and preprocess the image
image_path = 'CFD_001.jpg'
image = cv2.imread(image_path)
image = cv2.resize(image, (112, 112))  # Resize to match your model's input shape
# image = image / 255.0  # Normalize the pixel values
cv2.imwrite( "asg.jpg", image)
# Convert the image to a NumPy array with the expected shape
# input_image = np.expand_dims(image, axis=0)  # Add a batch dimension
prediction = model.predict(input_image)
prediction

NameError: name 'input_image' is not defined

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

prediction *= 255

prediction_2d = prediction.squeeze()

plt.figure(figsize=(8, 8))
plt.imshow(prediction_2d, cmap='gray')
plt.colorbar()
plt.axis('off')
plt.show()

NameError: name 'prediction' is not defined